In [1]:
import os 

In [2]:
os.chdir('../')

In [3]:
%pwd

'/Users/hitam/Downloads/TextSummarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)

class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
            
        )

        return data_transformation_config

In [7]:
import os
import urllib.request as request
from urllib.request import urlretrieve
import zipfile
from src.textsummarizer.logging import logger 
from src.textsummarizer.utils.common import get_size
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/Users/hitam/Downloads/TextSummarizer/texts/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [8]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)



    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-12-07 16:08:22,629: INFO: common: Yaml File : config/config.yaml loaded successfully]
[2023-12-07 16:08:22,634: INFO: common: Yaml File : params.yaml loaded successfully]
[2023-12-07 16:08:22,638: INFO: common: created directory at path: artifacts]
[2023-12-07 16:08:22,646: INFO: common: created directory at path: artifacts/data_transformation]


tokenizer_config.json: 100%|██████████| 88.0/88.0 [00:00<00:00, 7.08kB/s]
config.json: 100%|██████████| 1.12k/1.12k [00:00<00:00, 287kB/s]
spiece.model: 100%|██████████| 1.91M/1.91M [00:00<00:00, 6.91MB/s]
special_tokens_map.json: 100%|██████████| 65.0/65.0 [00:00<00:00, 14.6kB/s]
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/hitam/Downloads/TextSummarizer/texts/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 22692.01 examples/s]
